# Oyster Pathogen WRI Project
## This is a notebook for analyzing Nanopore long read amplicons for the WRI Oyster Pathogen Project
### E Suter Feb 2023

Data were produced in Minknow. Fastq files generated using guppy basecalling (using "Accurate" mode) and demultiplexed (12 barcodes)

Follow [this link](https://faisalmalikwp.medium.com/install-and-configure-jupyter-lab-using-miniconda-on-ubuntu-18-04-b7abd3f411a8) for setting up jupyter labs

Follow [this link](http://slhogle.github.io/2017/bash_jupyter_notebook/) for setting up the bash kernel in Jupyter.  
Simply used `pip install bash_kernel`
`python -m bash_kernel.install`

To launch notebook:

```conda activate nanopore```  
```jupyter lab```

`nanopore` is the name I chose for my environment. Can be whatever you named it

In [1]:
# once inside notebook, activate again
conda activate nanopore

(nanopore) 


: 1

### Use Nanoplot to view summary of sequencing stats

Install Nanoplot from [https://github.com/wdecoster/NanoPlot](https://github.com/wdecoster/NanoPlot).  
[Reference for Nanoplot](https://academic.oup.com/bioinformatics/article/34/15/2666/4934939?login=false)

In [6]:
#pip install Nanoplot

  Using cached NanoPlot-1.41.0.tar.gz (31 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 4.3 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 6.1 MB/s eta 0:00:00m eta 0:00:010:01:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 6.2 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 6.1 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 5.6 MB/s eta 0:00:00m eta 0:00:010:00:01
  Using cached nanoget-1.18.1.tar.gz (24 kB)
  Preparing metadata (setup.py) ... done
  Using cached nanomath-1.2.1.tar.gz (17 kB)
  Preparing metadata (setup.py) ... done
  Using cached plotly-5.13.0-py2.py3-none-any.whl (15.2 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 5.6 MB/s eta 0:00:00m eta 0:00:010:00:01
  Using cached kaleido-0.2.1-py2.py3-none-ma

In [ ]:
# NanoPlot --summary raw_data/sequencing_summary.txt --loglength -o summary-plots-log-transformed

The above outputs a bunch of figures in `summary-plots-log-transformed/`.  
For example:

![title](summary-plots-log-transformed/LengthvsQualityScatterPlot_dot.png)

The above shows most of my reads are around 4.2kB as expected and the average quality is about 13-14. There is a peak of reads at ~8.4kB, twice the amplicone length, so these are probablly chimeras. Same for ~12.6kB. Use `Nanofilt` to trim.  


### Use Nanofilt to quality filter reads

`Nanofilt` is available from [here](https://github.com/wdecoster/nanofilt)  
Note- `Nanofilt` soon will be replaced with `chopper`? Keep checking

In [7]:
# pip install nanofilt

  Using cached NanoFilt-2.8.0.tar.gz (7.4 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for nanofilt: filename=NanoFilt-2.8.0-py3-none-any.whl size=8781 sha256=1b46647c8e6d7187cff40a0717aca06c26027dd257ed76342f0865b478b97e24
  Stored in directory: /home/esuter/.cache/pip/wheels/53/4b/76/3e301955a8a693c62837d0be92909b33d93e40f5de21555e0b
Successfully built nanofilt


In [2]:
# First concatenate all samples from same barcode into one fastq file
#  concatenate content of all fastq files (I think each represents one pore) within each folder into one fastq file per sample/barcode
cd raw_data/barcode01
cat *.fastq.gz > b01.fq.gz
cd ..

cd barcode02
cat *.fastq.gz > b02.fq.gz
cd ..

cd barcode03
cat *.fastq.gz > b03.fq.gz
cd ..

cd barcode04
cat *.fastq.gz > b04.fq.gz
cd ..

cd barcode05
cat *.fastq.gz > b05.fq.gz
cd ..

cd barcode06
cat *.fastq.gz > b06.fq.gz
cd ..

cd barcode07
cat *.fastq.gz > b07.fq.gz
cd ..

cd barcode08
cat *.fastq.gz > b08.fq.gz
cd ..

cd barcode09
cat *.fastq.gz > b09.fq.gz
cd ..

cd barcode10
cat *.fastq.gz > b10.fq.gz
cd ..

cd barcode11
cat *.fastq.gz > b11.fq.gz
cd ..

cd barcode12
cat *.fastq.gz > b12.fq.gz
cd ..

In [4]:
# move concatenated files up to raw_data directory
cd ..
mv raw_data/barcode*/b* raw_data

In [5]:
# make directory for trimmed sequences
mkdir trimmed_data

In [8]:
# For each barcode, unzip, trim reads with quality score less than 10, make keep lengths +/- 20% of expected length (3360-5040). 
# Trim first 50 bp which are likely primer + tail
# Rezip and put into trimmed_data directory
gunzip -c raw_data/b01.fq.gz | NanoFilt -q 10 -l 3360 --maxlength 5040 --headcrop 50 | gzip > trimmed_data/b01.fq.gz

In [9]:
gunzip -c raw_data/b02.fq.gz | NanoFilt -q 10 -l 3360 --maxlength 5040 --headcrop 50 | gzip > trimmed_data/b02.fq.gz
gunzip -c raw_data/b03.fq.gz | NanoFilt -q 10 -l 3360 --maxlength 5040 --headcrop 50 | gzip > trimmed_data/b03.fq.gz
gunzip -c raw_data/b04.fq.gz | NanoFilt -q 10 -l 3360 --maxlength 5040 --headcrop 50 | gzip > trimmed_data/b04.fq.gz
gunzip -c raw_data/b05.fq.gz | NanoFilt -q 10 -l 3360 --maxlength 5040 --headcrop 50 | gzip > trimmed_data/b05.fq.gz
gunzip -c raw_data/b06.fq.gz | NanoFilt -q 10 -l 3360 --maxlength 5040 --headcrop 50 | gzip > trimmed_data/b06.fq.gz
gunzip -c raw_data/b07.fq.gz | NanoFilt -q 10 -l 3360 --maxlength 5040 --headcrop 50 | gzip > trimmed_data/b07.fq.gz
gunzip -c raw_data/b08.fq.gz | NanoFilt -q 10 -l 3360 --maxlength 5040 --headcrop 50 | gzip > trimmed_data/b08.fq.gz
gunzip -c raw_data/b09.fq.gz | NanoFilt -q 10 -l 3360 --maxlength 5040 --headcrop 50 | gzip > trimmed_data/b09.fq.gz
gunzip -c raw_data/b10.fq.gz | NanoFilt -q 10 -l 3360 --maxlength 5040 --headcrop 50 | gzip > trimmed_data/b10.fq.gz
gunzip -c raw_data/b11.fq.gz | NanoFilt -q 10 -l 3360 --maxlength 5040 --headcrop 50 | gzip > trimmed_data/b11.fq.gz
gunzip -c raw_data/b12.fq.gz | NanoFilt -q 10 -l 3360 --maxlength 5040 --headcrop 50 | gzip > trimmed_data/b12.fq.gz

In [11]:
# Use Nanoplot again to visualize reads now. No longer can use summary file (that was output from Minknow). Instead use fastq files directly:
NanoPlot -t 2 --fastq trimmed_data/*.fq.gz --plots hex dot

In [18]:
# Move output from above into a directory
mkdir trimmed_data_summary-plots
mv *.html trimmed_data_summary-plots/
mv *.png trimmed_data_summary-plots/
mv *.log trimmed_data_summary-plots/
mv NanoStats.txt trimmed_data_summary-plots/

mv: cannot stat '*.html': No such file or directory
mv: cannot stat '*.png': No such file or directory
mv: cannot stat '*.log': No such file or directory


Look at plots again:

![title](trimmed_data_summary-plots/LengthvsQualityScatterPlot_dot.png)

### Annotation

In [24]:
# Install ncbi toolkit
# run in terminal because need to put in password
## sudo apt-get install ncbi-blast+

[sudo] password for esuter: 


In [25]:
# check
blastn -h

USAGE
  blastn [-h] [-help] [-import_search_strategy filename]
    [-export_search_strategy filename] [-task task_name] [-db database_name]
    [-dbsize num_letters] [-gilist filename] [-seqidlist filename]
    [-negative_gilist filename] [-entrez_query entrez_query]
    [-db_soft_mask filtering_algorithm] [-db_hard_mask filtering_algorithm]
    [-subject subject_input_file] [-subject_loc range] [-query input_file]
    [-out output_file] [-evalue evalue] [-word_size int_value]
    [-gapopen open_penalty] [-gapextend extend_penalty]
    [-perc_identity float_value] [-qcov_hsp_perc float_value]
    [-max_hsps int_value] [-xdrop_ungap float_value] [-xdrop_gap float_value]
    [-xdrop_gap_final float_value] [-searchsp int_value]
    [-sum_stats bool_value] [-penalty penalty] [-reward reward] [-no_greedy]
    [-min_raw_gapped_score int_value] [-template_type type]
    [-template_length int_value] [-dust DUST_options]
    [-filtering_db filtering_database]
    [-window_masker_taxid window_ma

According to [this](https://stackoverflow.com/questions/70370949/local-blast-ncbi-c-exception), the blastn sudo-apt hasn't been updated since version 2.5 in 2017 and leads to errors (which I began having below). 


Update to latest, v2.13.0 

In [68]:
# conda install blast=2.13.0

: 130

In [6]:
blastn -version

blastn: 2.13.0+
 Package: blast 2.13.0, build Jul 18 2022 22:49:37
(nanopore) 


: 1

In [ ]:
# make new directory for fasta files
# mkdir trimmed_fasta

In [ ]:
# unzip fq files for input to fasta
#gunzip trimmed_data/*.fq.gz trimmed_data/*.fq

install [pyfastx](https://pypi.org/project/pyfastx/) for handling fastq files

In [ ]:
# pip install pyfastx

In [51]:
# convert fastq to fasta files
pyfastx fq2fa -o trimmed_fasta/b01.fa trimmed_data/b01.fq

In [52]:
pyfastx fq2fa -o trimmed_fasta/b02.fa trimmed_data/b02.fq
pyfastx fq2fa -o trimmed_fasta/b03.fa trimmed_data/b03.fq
pyfastx fq2fa -o trimmed_fasta/b04.fa trimmed_data/b04.fq
pyfastx fq2fa -o trimmed_fasta/b05.fa trimmed_data/b05.fq
pyfastx fq2fa -o trimmed_fasta/b06.fa trimmed_data/b06.fq
pyfastx fq2fa -o trimmed_fasta/b07.fa trimmed_data/b07.fq
pyfastx fq2fa -o trimmed_fasta/b08.fa trimmed_data/b08.fq
pyfastx fq2fa -o trimmed_fasta/b09.fa trimmed_data/b09.fq
pyfastx fq2fa -o trimmed_fasta/b10.fa trimmed_data/b10.fq
pyfastx fq2fa -o trimmed_fasta/b11.fa trimmed_data/b11.fq
pyfastx fq2fa -o trimmed_fasta/b12.fa trimmed_data/b12.fq

Test blast tools for annotation. Start with smallest file first, b11

Ibironke et al. use discontinuous mega-blast (`dc-megablast` from the blast [docs](https://www.ncbi.nlm.nih.gov/books/NBK569839/)) but this is depracated. Seems to be incorporated into blastn?

In [53]:
mkdir results 

The headings of the output table will be the following parameters from NCBI:
- seqid: query (e.g., unknown gene) sequence id
- sseqid: subject (e.g., reference genome) sequence id
- pident: percentage of identical matches
- length: alignment length (sequence overlap)
- mismatch: number of mismatches
- evalue: expect value
- bitscore: bit score
- staxids: Subject Taxonomy ID(s), separated by a ‘;’
- stitle: Subject Title

According to [this](https://www.metagenomics.wiki/tools/blast/megablast) `task-blastn` should be used for finding different species, `task-megablast` should be used for finding differents within the same species (and thus is faster), but seems like I should be using regular blastn

In [11]:
# test blastn on small subset i copied from b12 (pos ctl)

blastn -query trimmed_fasta/test.fa \
-db nt \
-out results/test_blast.txt \
-remote \
-max_target_seqs 1 \
-task dc-megablast \
-outfmt "6 qseqid sseqid pident length mismatch evalue bitscore staxids stitle" 

(nanopore) 
(nanopore) 

(nanopore) 


<i> After running the test file, I see there are multiple species output for one input sequence. It seems like it gives the top 2-3 hits. Need to do some post filtering to get one-for-one in species table. Not sure why it's doing this because I set max target seqs to 1 </i>

The warning about 5 or more matches is related to the -max_target_seqs, I think

In [3]:
# barcode 1
blastn -query trimmed_fasta/b01.fa \
-db nt \
-out results/b01.txt \
-remote \
-max_target_seqs 1 \
-outfmt "6 qseqid sseqid pident length mismatch evalue bitscore staxids stitle" 


(nanopore) 


In [ ]:
blastn -query trimmed_fasta/b02.fa \
-db nt \
-out results/b02.txt \
-remote \
-max_target_seqs 1 \
-task-blastn \
-outfmt "6 qseqid sseqid pident length mismatch evalue bitscore staxids stitle" 

blastn -query trimmed_fasta/b03.fa \
-db nt \
-out results/b03.txt \
-remote \
-max_target_seqs 1 \
-task-blastn \
-outfmt "6 qseqid sseqid pident length mismatch evalue bitscore staxids stitle" 

blastn -query trimmed_fasta/b04.fa \
-db nt \
-out results/b04.txt \
-remote \
-max_target_seqs 1 \
-task-blastn \
-outfmt "6 qseqid sseqid pident length mismatch evalue bitscore staxids stitle" 

blastn -query trimmed_fasta/b05.fa \
-db nt \
-out results/b05.txt \
-remote \
-max_target_seqs 1 \
-task-blastn \
-outfmt "6 qseqid sseqid pident length mismatch evalue bitscore staxids stitle" 

blastn -query trimmed_fasta/b06.fa \
-db nt \
-out results/b06.txt \
-remote \
-max_target_seqs 1 \
-task-blastn \
-outfmt "6 qseqid sseqid pident length mismatch evalue bitscore staxids stitle" 

blastn -query trimmed_fasta/b07.fa \
-db nt \
-out results/b07.txt \
-remote \
-max_target_seqs 1 \
-task-blastn \
-outfmt "6 qseqid sseqid pident length mismatch evalue bitscore staxids stitle" 

blastn -query trimmed_fasta/b08.fa \
-db nt \
-out results/b08.txt \
-remote \
-max_target_seqs 1 \
-task-blastn \
-outfmt "6 qseqid sseqid pident length mismatch evalue bitscore staxids stitle" 

blastn -query trimmed_fasta/b09.fa \
-db nt \
-out results/b09.txt \
-remote \
-max_target_seqs 1 \
-task-blastn \
-outfmt "6 qseqid sseqid pident length mismatch evalue bitscore staxids stitle" 

blastn -query trimmed_fasta/b10.fa \
-db nt \
-out results/b10.txt \
-remote \
-max_target_seqs 1 \
-task-blastn \
-outfmt "6 qseqid sseqid pident length mismatch evalue bitscore staxids stitle" 

blastn -query trimmed_fasta/b11.fa \
-db nt \
-out results/b11.txt \
-remote \
-max_target_seqs 1 \
-task-blastn \
-outfmt "6 qseqid sseqid pident length mismatch evalue bitscore staxids stitle" 

blastn -query trimmed_fasta/b12.fa \
-db nt \
-out results/b12.txt \
-remote \
-max_target_seqs 1 \
-task-blastn \
-outfmt "6 qseqid sseqid pident length mismatch evalue bitscore staxids stitle" 

### Try MiRROR
Abandon BLAST for now.
Found [this relatively new pub](https://journals.asm.org/doi/10.1128/spectrum.02017-21)

Follow instructions for use from [github repo](https://github.com/seoldh/MIrROR) and [website](http://mirror.egnome.co.kr/)


In [37]:
conda install numpy

  Using cached https://files.pythonhosted.org/packages/3a/5f/47e578b3ae79e2624e205445ab77a1848acdaa2929a00eeef6b16eaaeb20/numpy-1.16.6-cp27-cp27mu-manylinux1_x86_64.whl
(nanopore2) 


: 1

In [6]:
## had to make new environment to get these installs to work
# conda create --name nanopore2
# conda activate nanopore2

## For dependencies

# conda install -c bioconda minimap2      # long-read mapper
# conda install -c bioconda krona rename  # Krona plot
pip install pandas matplotlib           # Stacked bar plot

minimap2 --version
ktImportTaxonomy # just check one of Krona functions



(nanopore2) 
(nanopore2) 
(nanopore2) 
(nanopore2) 
(nanopore2) 
(nanopore2) 
(nanopore2) 
(nanopore2) 
  Using cached pandas-1.5.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.0 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 6.2 MB/s eta 0:00:00m eta 0:00:010:00:01
  Using cached python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)
  Using cached pytz-2022.7.1-py2.py3-none-any.whl (499 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.0/300.0 kB 6.6 MB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.4/965.4 kB 6.0 MB/s eta 0:00:006.3 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 6.3 MB/s eta 0:00:006.4 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 6.5 MB/s eta 0:00:000:00:010:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.3/98.3 kB 6.2 MB/s eta 0:00:00
  Using cached six-1.

: 1

In [20]:
## Getting Started
# git clone https://github.com/seoldh/MIrROR.git
# cd MIrROR
# ./MIrROR.py -h

(nanopore2) 
Cloning into 'MIrROR'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 49 (delta 0), reused 3 (delta 0), pack-reused 46
Receiving objects: 100% (49/49), 49.31 MiB | 3.26 MiB/s, done.
Resolving deltas: 100% (9/9), done.
(nanopore2) 
(nanopore2) 
Usage: MIrROR.py [options]  (-d DBDIR) INPUTFILE

MIrROR: a tool for metataxonomics with 16S-23S rRNA operon region.
INPUTFILE: FASTA/FASTQ/PAF | SAMPLELIST

Options:
  -d DBDIR, --db_dir=DBDIR
                        directory containing MIrROR database [required]
  -o OUTDIR, --output_dir=OUTDIR
                        specify directory to output files (default : ./Result)
  -t INT, --threads=INT
                        number of threads (default : 4)

  Mapping option:
    this option control for Mapping

    -M NUM, --minibatch=NUM
                        number of query bases loaded to the memory at once.
                        K/

: 1

In [21]:
## Download 16S-23S rRNA operon database
#mkdir DBDIR
#wget -P DBDIR http://mirror.egnome.co.kr/media/ToolsDatabase/2021_06/MIrROR_DB_r01.mmi
#wget -P DBDIR http://mirror.egnome.co.kr/media/ToolsDatabase/2021_06/MIrROR_DB_r01.tsv

(nanopore2) 
(nanopore2) 
--2023-02-10 12:30:09--  http://mirror.egnome.co.kr/media/ToolsDatabase/2021_06/MIrROR_DB_r01.mmi
Resolving mirror.egnome.co.kr (mirror.egnome.co.kr)... 147.47.67.225
Connecting to mirror.egnome.co.kr (mirror.egnome.co.kr)|147.47.67.225|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 846091773 (807M) [application/octet-stream]
Saving to: ‘DBDIR/MIrROR_DB_r01.mmi’

MIrROR_DB_r01.mmi   100%[===================>] 806.90M   224KB/s    in 1h 46m  

2023-02-10 14:16:17 (130 KB/s) - ‘DBDIR/MIrROR_DB_r01.mmi’ saved [846091773/846091773]

(nanopore2) 
--2023-02-10 14:16:17--  http://mirror.egnome.co.kr/media/ToolsDatabase/2021_06/MIrROR_DB_r01.tsv
Resolving mirror.egnome.co.kr (mirror.egnome.co.kr)... 147.47.67.225
Connecting to mirror.egnome.co.kr (mirror.egnome.co.kr)|147.47.67.225|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://mirror.egnome.co.kr/003892724856/media/ToolsDatabase/2021_06/MIrROR_DB_r01.tsv

: 1

In [7]:
# Try on test data
./MIrROR.py -V -d DBDIR -o ../results/test ../trimmed_fasta/test.fa

(nanopore2) 
MIrROR analysis tool v1.0 Built on Apr 28 2022
Command                    : ./MIrROR.py -V -d DBDIR -o ../results/test ../trimmed_fasta/test.fa
DataBase                   : /media/esuter/MyPassport/Oyster_Pathogen_WRI_Jupyter/MIrROR/DBDIR/MIrROR_DB_r01.mmi
                             /media/esuter/MyPassport/Oyster_Pathogen_WRI_Jupyter/MIrROR/DBDIR/MIrROR_DB_r01.tsv
                             Database release 01 (built Jan 21 2021)
Input Format               : [FASTA] /media/esuter/MyPassport/Oyster_Pathogen_WRI_Jupyter/trimmed_fasta/test.fa
Group information          : No
Read filtering threshold   : Number of residue matches - 2500 bp, Alignment block length - 3500 bp


[2023-02-10 15:08:26] INFO : MIrROR analysis Start
[2023-02-10 15:08:26] INFO : filecheck STEP START
[2023-02-10 15:08:26] INFO : 1 sample proceeding
[2023-02-10 15:08:26] INFO : mapping STEP START
[2023-02-10 15:08:26] INFO : 1 / 1 Mapping...
[2023-02-10 15:08:29] INFO : classification STEP START
[202

: 1

Got it working. Run the samples. Still need to find out alignment quality- that is hidden somewhere in .paf file (output from minimap2). But at least generate tables for now

In [8]:
./MIrROR.py -V -d DBDIR -o ../results/b01 ../trimmed_fasta/b01.fa
./MIrROR.py -V -d DBDIR -o ../results/b02 ../trimmed_fasta/b02.fa
./MIrROR.py -V -d DBDIR -o ../results/b03 ../trimmed_fasta/b03.fa
./MIrROR.py -V -d DBDIR -o ../results/b04 ../trimmed_fasta/b04.fa
./MIrROR.py -V -d DBDIR -o ../results/b05 ../trimmed_fasta/b05.fa
./MIrROR.py -V -d DBDIR -o ../results/b06 ../trimmed_fasta/b06.fa
./MIrROR.py -V -d DBDIR -o ../results/b07 ../trimmed_fasta/b07.fa
./MIrROR.py -V -d DBDIR -o ../results/b08 ../trimmed_fasta/b08.fa
./MIrROR.py -V -d DBDIR -o ../results/b09 ../trimmed_fasta/b09.fa
./MIrROR.py -V -d DBDIR -o ../results/b10 ../trimmed_fasta/b10.fa
./MIrROR.py -V -d DBDIR -o ../results/b11 ../trimmed_fasta/b11.fa
./MIrROR.py -V -d DBDIR -o ../results/b12 ../trimmed_fasta/b12.fa

MIrROR analysis tool v1.0 Built on Apr 28 2022
Command                    : ./MIrROR.py -V -d DBDIR -o ../results/b01 ../trimmed_fasta/b01.fa
DataBase                   : /media/esuter/MyPassport/Oyster_Pathogen_WRI_Jupyter/MIrROR/DBDIR/MIrROR_DB_r01.mmi
                             /media/esuter/MyPassport/Oyster_Pathogen_WRI_Jupyter/MIrROR/DBDIR/MIrROR_DB_r01.tsv
                             Database release 01 (built Jan 21 2021)
Input Format               : [FASTA] /media/esuter/MyPassport/Oyster_Pathogen_WRI_Jupyter/trimmed_fasta/b01.fa
Group information          : No
Read filtering threshold   : Number of residue matches - 2500 bp, Alignment block length - 3500 bp


[2023-02-10 15:17:55] INFO : MIrROR analysis Start
[2023-02-10 15:17:55] INFO : filecheck STEP START
[2023-02-10 15:17:55] INFO : 1 sample proceeding
[2023-02-10 15:17:55] INFO : mapping STEP START
[2023-02-10 15:17:55] INFO : 1 / 1 Mapping...
[2023-02-10 16:36:01] INFO : classification STEP START
[2023-02-10 16:36:01

: 1

### Move to R for analysis of OTU tables, etc